<a href="https://colab.research.google.com/github/arathee2/black-mamba/blob/Juan/Solar_PV_in_Aerial_Imagery_IDS_705.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
from tensorflow.keras.optimizers import RMSprop


# Google Drive Connection
from google.colab import drive
drive.mount('/content/drive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Which GPU do I have?

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Feb 14 22:46:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
# Load Kaggle images

!ls "/content/drive/My Drive/kaggle" 

'''
Set directory parameters
'''
dir_train_images  = '/content/drive/My Drive/kaggle/training/'
dir_test_images   = '/content/drive/My Drive/kaggle/testing/'
dir_train_labels  = '/content/drive/My Drive/kaggle/labels_training.csv'
dir_test_ids      = '/content/drive/My Drive/kaggle/sample_submission.csv'

labels_training.csv	 submission_02142020_2.csv  testing
sample_submission.csv	 submission_02142020_3.csv  training
solar_classification.py  submission_02142020.csv


In [0]:
def load_data(dir_data, dir_labels, training=True):
    ''' Load each of the image files into memory 

    While this is feasible with a smaller dataset, for larger datasets,
    not all the images would be able to be loaded into memory

    When training=True, the labels are also loaded
    '''
    labels_pd = pd.read_csv(dir_labels)
    ids       = labels_pd.id.values
    data      = []
    for identifier in ids:
        fname     = dir_data + identifier.astype(str) + '.tif'
        image     = mpl.image.imread(fname)
        data.append(image)
    data = np.array(data) # Convert to Numpy array
    if training:
        labels = labels_pd.label.values
        return data, labels
    else:
        return data, ids

In [0]:
# Split available images in training and validation
images, labels = load_data(dir_train_images, dir_train_labels, training=True)

training_images, validation_images, training_labels, validation_labels = train_test_split(images, labels, test_size=0.3)
test_images, test_ids = load_data(dir_test_images, dir_test_ids, training=False)

In [6]:
print(training_images.shape)
print(training_labels.shape)
print(validation_images.shape)
print(validation_labels.shape)
print(test_images.shape)
print(test_ids.shape)

(1050, 101, 101, 3)
(1050,)
(450, 101, 101, 3)
(450,)
(558, 101, 101, 3)
(558,)


In [0]:
training_images=training_images / 255.0
validation_images=validation_images / 255.0
test_images=test_images/255.0

In [13]:
# CNN - Network design

# Class to stop training when 99% accuracy is reached
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.999):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(101, 101, 3), kernel_regularizer = tf.keras.regularizers.l2(0.01)), # 64 convolutions or 'filters' each filter is 3x3
  tf.keras.layers.MaxPooling2D(2, 2), # Pooling with 2x2 matrix
# tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.01)), # 64 convolutions or 'filters' each filter is 3x3
# tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1024, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

# Training

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(training_images, training_labels, epochs=50, callbacks=[callbacks])
validation_loss = model.evaluate(validation_images, validation_labels)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 99, 99, 64)        1792      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 153664)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              157352960 
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 1025      
Total params: 157,355,777
Trainable params: 157,355,777
Non-trainable params: 0
_________________________________________________________________
Train on 1050 samples
Epoch 1/50
1050/1050 [==============================] - 1s 1ms/sample - loss: 4.2158

In [0]:
# Classify test data

test_predictions = model.predict(test_images)

In [0]:
submission = pd.DataFrame({"id":test_ids, "score":test_predictions[:,0]})
submission

submission.to_csv("/content/drive/My Drive/kaggle/submission_02142020_4.csv", index = False)